### Combine Award Sources to Create Single Awards Table

**Priority Order (lower = higher priority):**
- 0: GTR Project Awards (authoritative for UK grants, full metadata)
- 1: Crossref Awards (rich metadata from Crossref)
- 2: Backfill Awards (extracted from publication funding acknowledgements)
- 3: NIH Awards (US NIH grants with full metadata)
- 3: NSF Awards (US NSF grants with full metadata)
- 3: NSERC Awards (Canadian NSERC grants with full metadata)
- 3: GTR Awards (legacy publication-based, for work linkage only)
- 4: Gates Foundation Awards (Bill & Melinda Gates Foundation committed grants)
- 5: SSHRC Awards (Social Sciences and Humanities Research Council of Canada)
- 6: ANR Awards (Agence Nationale de la Recherche - French National Research Agency)
- 7: CIHR Awards (Canadian Institutes of Health Research)
- 8: CFI Awards (Canada Foundation for Innovation)
- 9: Vinnova Awards (Sweden's Innovation Agency)
- 10: KAKEN Awards (Japan Society for the Promotion of Science - Grants-in-Aid)
- 11: NWO Awards (Dutch Research Council)
- 12: NHMRC Awards (National Health and Medical Research Council - Australia)
- 13: NIHR Awards (National Institute for Health and Care Research - UK)
- 14: SNSF Awards (Swiss National Science Foundation)
- 15: Swedish Research Council Awards (Vetenskapsrådet)
- 16: Formas Awards (Swedish Research Council for Sustainable Development)
- 17: Forte Awards (Swedish Research Council for Health, Working Life and Welfare)
- 18: DFG Awards (Deutsche Forschungsgemeinschaft - German Research Foundation)
- 19: FAPESP Awards (Fundação de Amparo à Pesquisa do Estado de São Paulo - São Paulo Research Foundation)
- 20: ARC Awards (Australian Research Council)
- 21: ERC Awards (European Research Council)
- 22: Wellcome Trust Awards (Wellcome Trust - UK philanthropic funder)
- 23: NASA Awards (NASA - US space agency research grants)
- 24: Academy of Finland Awards (Research Council of Finland - Finnish research funder)
- 25: DOE Awards (U.S. Department of Energy research grants)
- 26: DOD Awards (U.S. Department of Defense research grants)

When the same award appears in multiple sources, the highest priority source wins.

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.openalex_awards
USING delta
AS
WITH award_aggregates AS (
  SELECT
    CAST(REPLACE(award.id, 'https://openalex.org/G', '') AS BIGINT) as award_id,
    TRANSFORM(
      COLLECT_LIST(DISTINCT id), 
      w -> CONCAT('https://openalex.org/W', w)
    ) as funded_outputs,
    COUNT(DISTINCT id) as funded_outputs_count
  FROM openalex.works.openalex_works
  LATERAL VIEW explode(awards) as award
  WHERE size(awards) > 0
  GROUP BY award.id
),
deduplicated_awards AS (
  SELECT 
    * EXCEPT(priority, row_num)
  FROM (
    SELECT 
      *,
      ROW_NUMBER() OVER (PARTITION BY id ORDER BY priority ASC) as row_num
    FROM openalex.awards.openalex_awards_raw
  )
  WHERE row_num = 1
)
SELECT 
  da.*,
  SLICE(COALESCE(aa.funded_outputs, ARRAY()), 1, 100) as funded_outputs,
  COALESCE(aa.funded_outputs_count, 0) as funded_outputs_count
FROM 
  deduplicated_awards da
  LEFT JOIN award_aggregates aa 
    ON da.id = aa.award_id